# EXTRACT - Extração dos Dados

Importação dos dados através da API da Telecom X. Os dados estão em formato **JSON**.

Bibliotecas utilizadas no processo de Extração

In [157]:
import pandas as pd
import numpy as np
import requests
import re

Buscando os dados 

In [158]:
# Url da base de dados
data_url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

response = requests.get(data_url)

if response.status_code == 200:
    json_file = response.json()
else:
    print(f"Bad response: status code: {response.status_code}")
    


In [159]:
# Criando o Dataframe
raw_data = pd.DataFrame(json_file)

# Primeira visualização
raw_data.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


O dataframe tem 4 colunas aninhadas que precisam ser tratadas para refletir as colunas do dicionário de dados.

In [160]:
# Criando uma lista com as colunas que precisam ser normalizadas
df = pd.json_normalize(json_file)
df.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Com os dados devidamente carregados em um Pandas DataFrame, agora vou fazer uma visão geral dos dados contidos nesse dataframe.
Utilizando o método **.info()** do Pandas, fica fácil ter uma apanhado geral do tipo de dados, nomes de colunas, número de linhas e colunas temos no dataframe.

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

Utilizando o método .shape para ter de forma clara a quantidade de linhas e colunas.

In [162]:
rows, columns = df.shape
print(f'linhas: {rows}, colunas: {columns}')

linhas: 7267, colunas: 21


Criei um arquivo .py com o dicionário dos dados das colunas do dataset, isso irá me auxiliar a entender melhor o propósito de cada coluna do dataframe e dar o tratamento apropriado.

In [163]:
from dict_data import dict_data

for key, value in dict_data.items():
    print(f"{key}: {value}")

customerID: número de identificação único de cada cliente
Churn: se o cliente deixou ou não a empresa
gender: gênero (masculino e feminino)
SeniorCitizen:  informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
Partner: se o cliente possui ou não um parceiro ou parceira
Dependents: se o cliente possui ou não dependentes
tenure:  meses de contrato do cliente
PhoneService: assinatura de serviço telefônico
MultipleLines: assisnatura de mais de uma linha de telefone
InternetService: assinatura de um provedor internet
OnlineSecurity: assinatura adicional de segurança online
OnlineBackup: assinatura adicional de backup online 
DeviceProtection: assinatura adicional de proteção no dispositivo
TechSupport: assinatura adicional de suporte técnico, menos tempo de espera
StreamingTV: assinatura de TV a cabo 
StreamingMovies: assinatura de streaming de filmes 
Contract: tipo de contrato
PaperlessBilling: se o cliente prefere receber online a fatura
PaymentMethod: forma de pagamen

## Transform - Transformação

Com os dados devidamente carregados e agora com as colunas identificadas através do dicionário de dados disponibilizado junto a API, iniciarei a limpeza dos dados 

#### Renomeando as colunas

O formato atual dos nomes de colunas no dataframe, está fora do padrão e irá dificultar a manipulação na hora de fazer os comandos, por isso com ajuda do dicionário de dados, estarei renomeando essas colunas para além de facilitar o processo de manipulação, padronizar os nomes das mesmas.

Abaixo, criei uma função que receberá o nome da coluna e retornará o nome em formato snakeCase.

In [164]:
#renomeando as colunas para facilitar

# Função para padronização dos nomes das colunas
def format_column_name(name: str) -> str:
    
    name = name.replace('.', '')  # remove pontos
    # insere _ antes de uma letra maiúscula que vem depois de uma letra minúscula ou número
    name = re.sub(r'(?<=[a-z0-9])(?=[A-Z])', '_', name)
    return name.lower()


In [165]:
# cria uma lista com o nome atual das colunas do dataframe
old_columns= df.columns.tolist() 

# Cria uma lista, utilizando o retorno da função map(), que recebe a função criada acima e a lista com o nome atual das colunas do dataframe e retorna uma nova lista com o nome padronizado das colunas.
new_columns= list(map(format_column_name, list(dict_data.keys())))

# Um dicionário utilizando a função zip(), onde a key é o nome antigo e o value, o nome novo
dict_columns = {key: value for key, value in zip(old_columns, new_columns)}
dict_columns

{'customerID': 'customer_id',
 'Churn': 'churn',
 'customer.gender': 'gender',
 'customer.SeniorCitizen': 'senior_citizen',
 'customer.Partner': 'partner',
 'customer.Dependents': 'dependents',
 'customer.tenure': 'tenure',
 'phone.PhoneService': 'phone_service',
 'phone.MultipleLines': 'multiple_lines',
 'internet.InternetService': 'internet_service',
 'internet.OnlineSecurity': 'online_security',
 'internet.OnlineBackup': 'online_backup',
 'internet.DeviceProtection': 'device_protection',
 'internet.TechSupport': 'tech_support',
 'internet.StreamingTV': 'streaming_tv',
 'internet.StreamingMovies': 'streaming_movies',
 'account.Contract': 'contract',
 'account.PaperlessBilling': 'paperless_billing',
 'account.PaymentMethod': 'payment_method',
 'account.Charges.Monthly': 'charges_monthly',
 'account.Charges.Total': 'charges_total'}

Com o dicionário de novos nomes pronto, é hora de renomear as colunas do dataframe.

In [166]:
df.rename(columns=dict_columns, inplace=True)

In [167]:
# Resultado
df.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Para me ajudar a entender melhor os dados, vou visualizar as colunas e a quantidade de dados distintos em cada uma, isso me ajudará a ver como as colunas estão distribuídas entre dados booleanos, categóricos e numéricos.

In [168]:
df.nunique()

customer_id          7267
churn                   3
gender                  2
senior_citizen          2
partner                 2
dependents              2
tenure                 73
phone_service           2
multiple_lines          3
internet_service        3
online_security         3
online_backup           3
device_protection       3
tech_support            3
streaming_tv            3
streaming_movies        3
contract                3
paperless_billing       2
payment_method          4
charges_monthly      1585
charges_total        6531
dtype: int64

Com essa visualização, é fácil identificar que das 21 colunas, 4 uma grande variedade de dados distintos e o restante, 17 colunas tem dados categóricos/ booleanos.

In [169]:
# Laço de repetição, para exibir as colunas com menos dados únicos(categóricas) e o tipo de dados que cada coluna tem.

for col in df.columns.tolist():
    if df[col].nunique() < 5:
        print(f'{col} => {df[col].unique()}')

churn => ['No' 'Yes' '']
gender => ['Female' 'Male']
senior_citizen => [0 1]
partner => ['Yes' 'No']
dependents => ['Yes' 'No']
phone_service => ['Yes' 'No']
multiple_lines => ['No' 'Yes' 'No phone service']
internet_service => ['DSL' 'Fiber optic' 'No']
online_security => ['No' 'Yes' 'No internet service']
online_backup => ['Yes' 'No' 'No internet service']
device_protection => ['No' 'Yes' 'No internet service']
tech_support => ['Yes' 'No' 'No internet service']
streaming_tv => ['Yes' 'No' 'No internet service']
streaming_movies => ['No' 'Yes' 'No internet service']
contract => ['One year' 'Month-to-month' 'Two year']
paperless_billing => ['Yes' 'No']
payment_method => ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


Conhecendo o tipo de dado que está presente em cada coluna com poucos dados únicos. A maioria das colunas tem dados binários do tipo **(YES/NO)**, quando não acompanhado dos valores **(No phone service, No internet service, ou ' ')**, uma coluna com binário numérico **(0/1)** e 04 colunas categóricas **(gender, internet_service, contract, payment_method)**.

Para padronizar os dados, estarei modificando as colunas com dados de valores **(YES/NO)** para **(1/0)**, preservando os valores diferentes como **NaN ou Not informed**. A preservação desses valores visa melhor análise pelos modelos de ML e para não tendenciar o resultado dos dados consistentes.

In [170]:
#Lista com valores atuais das colunas que serão modificados
key_words = ['No', 'Yes', '', 'No phone service', 'No internet service']

# Lista com valores que substituirão os dados nas colunas
new_data = [0, 1, np.nan, 'not informed', 'not informed']

# Lista vazia que armazenará os nomes das colunas que terão os dados formatados.
data_to_change = []

for col in df.columns.tolist(): # Itera por todas as colunas
    if df[col].nunique() < 5:   # seleciona as com menos de 5 valores únicos.
        for  _ in df[col].unique().tolist(): # Itera por uma listas das colunas com menos de 5 valores únicos.
            if _ in key_words and col != 'internet_service':  # Verifica se algum dado da coluna está na lista key_words
                data_to_change.append(col) # Adiciona o nome da coluna a lista data_to_change.
                
print('++++++++++++++++++++++++++++++++')
print('Colunas para formatar os dados')
data_to_change = set(data_to_change)        # mantem apenas os valores únicos da lista
print(data_to_change )
dict_data_to_change = dict(zip(key_words, new_data))
print("Dicionário para mudança dos dados")
dict_data_to_change

++++++++++++++++++++++++++++++++
Colunas para formatar os dados
{'device_protection', 'partner', 'churn', 'tech_support', 'phone_service', 'streaming_tv', 'multiple_lines', 'online_backup', 'dependents', 'paperless_billing', 'online_security', 'streaming_movies'}
Dicionário para mudança dos dados


{'No': 0,
 'Yes': 1,
 '': nan,
 'No phone service': 'not informed',
 'No internet service': 'not informed'}

Com o dicionário criado, vou iterar sobre os dados atuais observando suas contagem para posterior comparação, a fim de evitar mudança nos dados.

In [171]:
for _ in data_to_change:
    print(f'{_}: {df[_].value_counts()}')
    print('==========================')

device_protection: device_protection
No                     3195
Yes                    2491
No internet service    1581
Name: count, dtype: int64
partner: partner
No     3749
Yes    3518
Name: count, dtype: int64
churn: churn
No     5174
Yes    1869
        224
Name: count, dtype: int64
tech_support: tech_support
No                     3582
Yes                    2104
No internet service    1581
Name: count, dtype: int64
phone_service: phone_service
Yes    6560
No      707
Name: count, dtype: int64
streaming_tv: streaming_tv
No                     2896
Yes                    2790
No internet service    1581
Name: count, dtype: int64
multiple_lines: multiple_lines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64
online_backup: online_backup
No                     3182
Yes                    2504
No internet service    1581
Name: count, dtype: int64
dependents: dependents
No     5086
Yes    2181
Name: count, dtype: int64
paperless_bill

Com o dicionário dos dados que serão alterados pronto, utilizarei o método .replace para alterar os dados

In [172]:
df[list(data_to_change)] = df[list(data_to_change)].replace(dict_data_to_change)

/tmp/ipykernel_27655/3336162844.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[list(data_to_change)] = df[list(data_to_change)].replace(dict_data_to_change)


Verificando se os dados foram alterados sem alterar as quantidades.

In [173]:
for _ in data_to_change:
    print(f'{_}: {df[_].value_counts()}')
    print('==========================')

device_protection: device_protection
0               3195
1               2491
not informed    1581
Name: count, dtype: int64
partner: partner
0    3749
1    3518
Name: count, dtype: int64
churn: churn
0.0    5174
1.0    1869
Name: count, dtype: int64
tech_support: tech_support
0               3582
1               2104
not informed    1581
Name: count, dtype: int64
phone_service: phone_service
1    6560
0     707
Name: count, dtype: int64
streaming_tv: streaming_tv
0               2896
1               2790
not informed    1581
Name: count, dtype: int64
multiple_lines: multiple_lines
0               3495
1               3065
not informed     707
Name: count, dtype: int64
online_backup: online_backup
0               3182
1               2504
not informed    1581
Name: count, dtype: int64
dependents: dependents
0    5086
1    2181
Name: count, dtype: int64
paperless_billing: paperless_billing
1    4311
0    2956
Name: count, dtype: int64
online_security: online_security
0               36

In [174]:
for col in df.columns.tolist():
    if df[col].nunique() < 50:
        print(f'{col} => {df[col].unique()}')

churn => [ 0.  1. nan]
gender => ['Female' 'Male']
senior_citizen => [0 1]
partner => [1 0]
dependents => [1 0]
phone_service => [1 0]
multiple_lines => [0 1 'not informed']
internet_service => ['DSL' 'Fiber optic' 'No']
online_security => [0 1 'not informed']
online_backup => [1 0 'not informed']
device_protection => [0 1 'not informed']
tech_support => [1 0 'not informed']
streaming_tv => [1 0 'not informed']
streaming_movies => [0 1 'not informed']
contract => ['One year' 'Month-to-month' 'Two year']
paperless_billing => [1 0]
payment_method => ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


In [175]:
df.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,0.0,Female,0,1,1,9,1,0,DSL,...,1,0,1,1,0,One year,1,Mailed check,65.6,593.3
1,0003-MKNFE,0.0,Male,0,0,0,9,1,1,DSL,...,0,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.4
2,0004-TLHLJ,1.0,Male,0,0,0,4,1,0,Fiber optic,...,0,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1.0,Male,1,1,0,13,1,0,Fiber optic,...,1,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1.0,Female,1,1,0,3,1,0,Fiber optic,...,0,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.4


In [176]:
df.dtypes

customer_id           object
churn                float64
gender                object
senior_citizen         int64
partner                int64
dependents             int64
tenure                 int64
phone_service          int64
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
streaming_movies      object
contract              object
paperless_billing      int64
payment_method        object
charges_monthly      float64
charges_total         object
dtype: object

Todas as colunas alvo foram alteradas com sucesso para o padrão ideal, um pequeno detalhe para coluna **churn** que teve as strings vazias transformadas em **Nan** e o tipo da coluna alterado automáticamente para float, isso acontece pq o pandas não consegue tratar dados de colunas com NaN como inteiros. Isso pode ser corrigido mudando o tipo da coluna para o **inteiro nullable do Pandas (Int64)**.

In [177]:
df['churn'] = df['churn'].astype('Int64')
df['churn'].dtype

Int64Dtype()

Mudando o tipo colunas com dados já tratados para int ou category

In [178]:

for col in data_to_change:
    if len(df[col].unique()) == 2:
        df[col] = df[col].astype(np.int64)
    else:
        df[col] = df[col].astype('category')
    

In [179]:
df.dtypes

customer_id            object
churn                category
gender                 object
senior_citizen          int64
partner                 int64
dependents              int64
tenure                  int64
phone_service           int64
multiple_lines       category
internet_service       object
online_security      category
online_backup        category
device_protection    category
tech_support         category
streaming_tv         category
streaming_movies     category
contract               object
paperless_billing       int64
payment_method         object
charges_monthly       float64
charges_total          object
dtype: object

Visualizando as últimas colunas que ainda estão como object

In [180]:
last_type_objects = df.select_dtypes(include=['object']).columns.tolist()
last_type_objects

['customer_id',
 'gender',
 'internet_service',
 'contract',
 'payment_method',
 'charges_total']

In [181]:
df[last_type_objects].head()

,customer_id,gender,internet_service,contract,payment_method,charges_total
0,0002-ORFBO,Female,DSL,One year,Mailed check,593.3
1,0003-MKNFE,Male,DSL,Month-to-month,Mailed check,542.4
2,0004-TLHLJ,Male,Fiber optic,Month-to-month,Electronic check,280.85
3,0011-IGKFF,Male,Fiber optic,Month-to-month,Electronic check,1237.85
4,0013-EXCHZ,Female,Fiber optic,Month-to-month,Mailed check,267.4


Das coslunas do tipo **Object** que restaram, apenas customer_id e charges_total não são do tipo **category**

In [182]:
last_type_objects = last_type_objects[1:5]

In [183]:
last_type_objects

['gender', 'internet_service', 'contract', 'payment_method']

In [184]:
df[last_type_objects].head()

,gender,internet_service,contract,payment_method
0,Female,DSL,One year,Mailed check
1,Male,DSL,Month-to-month,Mailed check
2,Male,Fiber optic,Month-to-month,Electronic check
3,Male,Fiber optic,Month-to-month,Electronic check
4,Female,Fiber optic,Month-to-month,Mailed check


Normalizando os dados e definindo o tipo como **category**

In [185]:
df[last_type_objects] = df[last_type_objects].apply(lambda x: x.str.lower()).astype('category')

In [186]:
df.dtypes

customer_id            object
churn                category
gender               category
senior_citizen          int64
partner                 int64
dependents              int64
tenure                  int64
phone_service           int64
multiple_lines       category
internet_service     category
online_security      category
online_backup        category
device_protection    category
tech_support         category
streaming_tv         category
streaming_movies     category
contract             category
paperless_billing       int64
payment_method       category
charges_monthly       float64
charges_total          object
dtype: object

In [187]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
1623,2305-MRGLV,0,male,0,1,0,28,1,0,fiber optic,...,0,0,1,0,0,month-to-month,0,bank transfer (automatic),76.55,2065.4
5627,7683-CBDKJ,1,male,0,1,1,14,1,0,dsl,...,1,0,0,1,0,month-to-month,1,electronic check,65.45,937.6
134,0222-CNVPT,0,male,1,0,0,52,0,not informed,dsl,...,0,0,0,1,1,month-to-month,1,credit card (automatic),48.80,2555.05
2662,3707-GNWHM,1,male,0,0,0,1,1,1,fiber optic,...,0,0,0,0,0,month-to-month,1,mailed check,74.25,74.25
221,0330-BGYZE,0,male,0,1,0,60,1,1,fiber optic,...,1,1,0,1,1,one year,0,bank transfer (automatic),102.50,6157.6


Terminado a limpeza/Transformação das colunas com poucos dados únicos, olhando agora para as colunas restantes e o tipo de dados contidos nelas.


In [188]:
for col in df.columns.tolist():
    if df[col].nunique() > 5:
        print(f'{col} => {df[col].unique()}')

customer_id => ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
tenure => [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
charges_monthly => [65.6  59.9  73.9  ... 91.75 68.8  67.85]
charges_total => ['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']


O tipo da coluna **charges_total** está como object e seus dados aparentam ser do tipo string embora sejam números, possívelmente existem strings vazias ou numeros com caracteres especiais.

In [189]:
empty_strings= df.apply(lambda x: x.astype(str).str.strip() == '').sum()
empty_strings[empty_strings > 0]

charges_total    11
dtype: int64

In [190]:
df[df['charges_total'] == ' ']

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
975,1371-DWPAZ,0,female,0,1,1,0,0,not informed,dsl,...,1,1,1,1,0,two year,0,credit card (automatic),56.05,
1775,2520-SGTTA,0,female,0,1,1,0,1,0,no,...,not informed,not informed,not informed,not informed,not informed,two year,0,mailed check,20.00,
1955,2775-SEFEE,0,male,0,0,1,0,1,1,dsl,...,1,0,1,0,0,two year,1,bank transfer (automatic),61.90,
2075,2923-ARZLG,0,male,0,1,1,0,1,0,no,...,not informed,not informed,not informed,not informed,not informed,one year,1,mailed check,19.70,
2232,3115-CZMZD,0,male,0,0,1,0,1,0,no,...,not informed,not informed,not informed,not informed,not informed,two year,0,mailed check,20.25,
2308,3213-VVOLG,0,male,0,1,1,0,1,1,no,...,not informed,not informed,not informed,not informed,not informed,two year,0,mailed check,25.35,
2930,4075-WKNIU,0,female,0,1,1,0,1,1,dsl,...,1,1,1,1,0,two year,0,mailed check,73.35,
3134,4367-NUYAO,0,male,0,1,1,0,1,1,no,...,not informed,not informed,not informed,not informed,not informed,two year,0,mailed check,25.75,
3203,4472-LVYGI,0,female,0,1,1,0,0,not informed,dsl,...,0,1,1,1,0,two year,1,bank transfer (automatic),52.55,
4169,5709-LVOEQ,0,female,0,1,1,0,1,0,dsl,...,1,1,0,1,1,two year,0,mailed check,80.85,


Temos 11 linhas com strings vazias na coluna **charges_total**

Olhando para o dataset, pude observar que a coluna **charges_total** é o produto da multiplicação da coluna **ternure** pela coluna **charge_monthly** com uma pequena diferença que pode ser alguma taxa de serviço. 

Uma coisa em comum é que em todas as linhas de **charges_tota** com string vazia, a coluna **tenure** tem valor igual a 0(zero). Ou seja, pode ser um cliente novo, com contrato ainda não faturado, pois também não deixaram a empresa, como pode ser observado através da coluna **churn**.

In [191]:
sum(df['tenure'] == 0)

11

In [192]:
df[df['charges_total'] == ' '][['churn','tenure','charges_total']]

,churn,tenure,charges_total
975,0,0,
1775,0,0,
1955,0,0,
2075,0,0,
2232,0,0,
2308,0,0,
2930,0,0,
3134,0,0,
3203,0,0,
4169,0,0,


Os dados faltantes serão subistituídos pelo valor 0.

In [193]:
# Lista com os indíces em que aparece a (' ')
idx_empty_strings = df[df['charges_total'] == ' '].index.tolist()

# Substituindo por 0(zero).
df.loc[idx_empty_strings, 'charges_total'] = 0

# alterando o tipo da coluna para float
df['charges_total'] = df['charges_total'].astype(np.float64)

In [194]:
df.dtypes

customer_id            object
churn                category
gender               category
senior_citizen          int64
partner                 int64
dependents              int64
tenure                  int64
phone_service           int64
multiple_lines       category
internet_service     category
online_security      category
online_backup        category
device_protection    category
tech_support         category
streaming_tv         category
streaming_movies     category
contract             category
paperless_billing       int64
payment_method       category
charges_monthly       float64
charges_total         float64
dtype: object

In [195]:
df.sample(10)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
1572,2239-CFOUJ,0,male,0,1,1,1,1,0,no,...,not informed,not informed,not informed,not informed,not informed,one year,0,mailed check,20.40,20.40
6952,9575-IWCAZ,0,male,0,1,0,6,1,1,no,...,not informed,not informed,not informed,not informed,not informed,one year,0,mailed check,25.10,171.00
1647,2338-BQEZT,0,female,0,0,0,55,1,0,dsl,...,0,0,0,0,0,month-to-month,1,credit card (automatic),50.55,2832.75
4070,5569-OUICF,1,female,1,1,0,28,1,1,fiber optic,...,0,0,1,1,1,month-to-month,1,credit card (automatic),101.30,2812.20
7058,9717-WXVSB,NaN,male,0,0,0,1,1,0,no,...,not informed,not informed,not informed,not informed,not informed,month-to-month,0,mailed check,18.90,18.90
2325,3243-ZHOHY,0,female,0,0,0,16,1,0,no,...,not informed,not informed,not informed,not informed,not informed,one year,0,mailed check,20.10,296.15
1503,2160-GPFXD,0,male,0,1,1,54,1,1,dsl,...,1,0,1,0,0,two year,1,credit card (automatic),65.65,3566.70
701,1000-AJSLD,1,male,0,0,0,1,1,0,no,...,not informed,not informed,not informed,not informed,not informed,month-to-month,1,mailed check,20.10,20.10
3936,5378-IKEEG,1,female,0,0,0,1,0,not informed,dsl,...,0,0,0,1,0,month-to-month,1,electronic check,35.75,35.75
3393,4704-ERYFC,1,female,0,1,0,22,1,0,fiber optic,...,0,0,0,0,0,month-to-month,1,electronic check,69.25,1554.00


### Enriquecendo o dataset

Como parte do desafio, foi sugerido a criação da coluna contas_diarias, utilizando o faturamento mensal para calcular o valor diário, proporcionando uma visão mais detalhada do comportamento dos clientes ao longo do tempo.

In [196]:
df['charges_daily'] = df['charges_monthly'] / 30

In [197]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total,charges_daily
3451,4785-NKHCX,0,male,1,0,0,5,1,0,no,...,not informed,not informed,not informed,not informed,month-to-month,0,bank transfer (automatic),20.05,94.15,0.668333
5770,7880-XSOJX,0,male,0,0,0,4,0,not informed,dsl,...,0,1,0,0,month-to-month,0,mailed check,42.40,146.40,1.413333
2061,2900-PHPLN,0,female,1,1,0,70,1,0,no,...,not informed,not informed,not informed,not informed,two year,1,credit card (automatic),19.55,1462.05,0.651667
6526,8970-ANWXO,0,female,0,0,0,23,1,1,dsl,...,0,1,1,1,one year,1,mailed check,73.75,1756.60,2.458333
408,0582-AVCLN,0,female,0,0,0,38,1,0,no,...,not informed,not informed,not informed,not informed,two year,0,mailed check,20.30,743.05,0.676667


### Alterando a ordem das colunas

In [209]:
current_columns = df.columns.tolist()
current_columns[-1], current_columns[-3] = current_columns[-3], current_columns[-1]
current_columns[-1], current_columns[-2] = current_columns[-2], current_columns[-1] 

In [211]:
df = df[current_columns]

In [212]:
df.sample(5)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_daily,charges_monthly,charges_total
821,1166-PQLGG,0,female,0,1,1,72,1,0,no,...,not informed,not informed,not informed,not informed,two year,0,bank transfer (automatic),0.651667,19.55,1463.45
6232,8573-CGOCC,0,male,0,0,0,16,1,0,no,...,not informed,not informed,not informed,not informed,month-to-month,0,credit card (automatic),0.658333,19.75,294.90
792,1120-BMWUB,1,female,0,0,0,16,1,0,dsl,...,0,1,0,0,month-to-month,1,mailed check,1.796667,53.90,834.15
5343,7296-PIXQY,0,female,0,1,1,14,1,0,fiber optic,...,0,0,0,0,month-to-month,0,electronic check,2.340000,70.20,1046.50
7017,9659-ZTWSM,0,male,1,1,0,66,1,1,fiber optic,...,1,1,1,1,one year,1,bank transfer (automatic),3.578333,107.35,7051.95


# Load - Salvando o DataSet 
Após a limpeza e alterações dos dados, é hora de salvar o dataset,

Com a conclusão do ETL - Extract, Transform, agora só falta o L de load. Decidí criar um banco de dados local para salvar o dados já limpos, facilitando o desempenho e consultas.

In [213]:
from sqlalchemy import create_engine, MetaData, Table, inspect
engine = create_engine('sqlite:///clened_telecomX_db/teleconX_db.db')
# Salvando no banco de dados
df.to_sql('teleconX_origin', engine, index=False, if_exists='replace')

7267

In [214]:
print(inspect(engine).get_table_names())

['teleconX_origin']


### Localizando o dataset para português brasileiro

Como uma atividade proposta, vou traduzir as colunas e os dados que podem ser traduzidos sem comprometer a integridade dos dados

In [220]:
ptbr_columns = ['id_cliente', 'contrato_cancelado', 'sexo', 'igual_maior_65', 'tem_parceiro','tem_dependente', 'meses_contrato', 'servicos_telefonicos', 'multiplas_linhas', 'servicos_internet', 'seguranca_online','backup_online','seguro_protecao','suporte_tecnico','tem_tv_cabo', 'tem_streaming_filmes', 'tipo_contrato','fatura_online','forma_pagamento','custo_diario','valor_mensal','valor_total']

In [221]:
dict_ptbr = dict(zip(df.columns.tolist(), ptbr_columns))
dict_ptbr

{'customer_id': 'id_cliente',
 'churn': 'contrato_cancelado',
 'gender': 'sexo',
 'senior_citizen': 'igual_maior_65',
 'partner': 'tem_parceiro',
 'dependents': 'tem_dependente',
 'tenure': 'meses_contrato',
 'phone_service': 'servicos_telefonicos',
 'multiple_lines': 'multiplas_linhas',
 'internet_service': 'servicos_internet',
 'online_security': 'seguranca_online',
 'online_backup': 'backup_online',
 'device_protection': 'seguro_protecao',
 'tech_support': 'suporte_tecnico',
 'streaming_tv': 'tem_tv_cabo',
 'streaming_movies': 'tem_streaming_filmes',
 'contract': 'tipo_contrato',
 'paperless_billing': 'fatura_online',
 'payment_method': 'forma_pagamento',
 'charges_daily': 'custo_diario',
 'charges_monthly': 'valor_mensal',
 'charges_total': 'valor_total'}

fazendo uma cópia do dataframe para renomear colunas em ptbr

In [246]:
df_br = df.copy()

In [247]:
df_br.rename(columns=dict_ptbr, inplace=True)
df_br.sample(5)

,id_cliente,contrato_cancelado,sexo,igual_maior_65,tem_parceiro,tem_dependente,meses_contrato,servicos_telefonicos,multiplas_linhas,servicos_internet,...,seguro_protecao,suporte_tecnico,tem_tv_cabo,tem_streaming_filmes,tipo_contrato,fatura_online,forma_pagamento,custo_diario,valor_mensal,valor_total
1789,2542-HYGIQ,0,female,0,0,0,1,1,0,no,...,not informed,not informed,not informed,not informed,month-to-month,0,credit card (automatic),0.665000,19.95,19.95
398,0568-ONFPC,1,male,0,1,1,5,1,1,no,...,not informed,not informed,not informed,not informed,month-to-month,1,bank transfer (automatic),0.863333,25.90,135.00
6365,8749-CLJXC,0,male,0,0,0,1,1,0,no,...,not informed,not informed,not informed,not informed,month-to-month,0,mailed check,0.668333,20.05,20.05
1441,2073-QBVBI,0,female,0,1,0,41,0,not informed,dsl,...,0,1,0,0,one year,0,mailed check,1.181667,35.45,1391.65
499,0704-VCUMB,0,female,0,1,0,61,1,0,no,...,not informed,not informed,not informed,not informed,two year,0,credit card (automatic),0.675000,20.25,1278.80


traduzindo dados

In [248]:
to_translate = ['sexo', 'servicos_internet', 'tipo_contrato', 'forma_pagamento']

In [255]:
dict_to_translate = {
    'male': 'masculino', 
    'female': 'feminino',
    'dsl': 'dsl-cabo', 
    'fiber optic': 'fibra optica', 
    'no' : 'sem serviço',
    'month-to-month': 'mensal', 
    'two year': 'dois anos', 
    'one year': 'anual',
    'electronic check': 'cheque eletrônico', 
    'mailed check': 'cheque via correios', 
    'bank transfer (automatic)': 'tranferência bancária automática', 
    'credit card (automatic)': 'cartão de crédito automático'
    }

In [256]:
for col in to_translate:
    df_br[col] = df_br[col].replace(dict_to_translate)

/tmp/ipykernel_27655/680145906.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_br[col] = df_br[col].replace(dict_to_translate)


In [257]:
for col in to_translate:
    print(df_br[col].value_counts())

sexo
masculino    3675
feminino     3592
Name: count, dtype: int64
servicos_internet
fibra optica    3198
dsl-cabo        2488
sem serviço     1581
Name: count, dtype: int64
tipo_contrato
mensal       4005
dois anos    1743
anual        1519
Name: count, dtype: int64
forma_pagamento
cheque eletrônico                   2445
cheque via correios                 1665
tranferência bancária automática    1589
cartão de crédito automático        1568
Name: count, dtype: int64


In [258]:
df_br.sample(10)

,id_cliente,contrato_cancelado,sexo,igual_maior_65,tem_parceiro,tem_dependente,meses_contrato,servicos_telefonicos,multiplas_linhas,servicos_internet,...,seguro_protecao,suporte_tecnico,tem_tv_cabo,tem_streaming_filmes,tipo_contrato,fatura_online,forma_pagamento,custo_diario,valor_mensal,valor_total
7140,9823-EALYC,0,masculino,0,1,1,72,1,1,dsl-cabo,...,1,1,1,0,dois anos,1,tranferência bancária automática,2.695000,80.85,5727.45
238,0361-HJRDX,0,feminino,0,0,0,68,0,not informed,dsl-cabo,...,1,1,1,1,dois anos,0,cartão de crédito automático,2.145000,64.35,4539.60
2421,3384-CTMSF,0,masculino,0,1,0,47,1,1,fibra optica,...,1,0,1,1,mensal,1,cheque eletrônico,3.470000,104.10,5135.15
2449,3417-TSCIC,0,masculino,0,0,0,29,0,not informed,dsl-cabo,...,0,0,0,0,anual,1,cheque via correios,0.828333,24.85,788.05
1514,2180-DXNEG,1,feminino,0,0,0,12,1,0,fibra optica,...,1,0,1,1,mensal,1,cheque eletrônico,3.200000,96.00,1062.10
4317,5930-GBIWP,0,masculino,0,0,0,69,1,1,dsl-cabo,...,1,1,0,1,dois anos,1,cartão de crédito automático,2.716667,81.50,5553.25
2660,3705-PSNGL,1,masculino,0,0,0,45,1,0,sem serviço,...,not informed,not informed,not informed,not informed,anual,1,cheque eletrônico,0.680000,20.40,930.45
1173,1676-MQAOA,0,masculino,0,0,0,72,1,0,dsl-cabo,...,1,1,1,0,dois anos,1,tranferência bancária automática,2.503333,75.10,5336.35
1590,2260-USTRB,1,feminino,1,0,0,2,1,0,fibra optica,...,0,0,0,0,mensal,1,tranferência bancária automática,2.340000,70.20,115.95
2066,2908-WGAXL,0,feminino,0,1,1,56,1,1,sem serviço,...,not informed,not informed,not informed,not informed,dois anos,1,cartão de crédito automático,0.831667,24.95,1468.90
